In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import numpy as np
import gc
import os
import optuna
import sqlite3
import ray
import matplotlib.pyplot as plt
from optuna.integration import LightGBMPruningCallback
from autogluon.tabular import TabularPredictor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Rutas de entrada y salida
input_path = './data/l_vm_completa_normalizada_fe.parquet'
output_train = './data/df_train.parquet'
output_val = './data/df_val.parquet'

# Periodos para división
periodo_train_max = 201908
periodos_val = [201909, 201910]

In [2]:
output_train_limpio = './data/df_train_limpio.parquet'
output_val_limpio = './data/df_val_limpio.parquet'

df_train_limpio = pd.read_parquet(output_train_limpio, engine='fastparquet')
df_val_limpio = pd.read_parquet(output_val_limpio, engine='fastparquet')

In [ ]:
from autogluon.tabular import TabularPredictor

predictor = TabularPredictor(
    label='CLASE_DELTA_ZSCORE',
    problem_type='regression',
    eval_metric='mean_absolute_error',
    verbosity=4,
    path='AutogluonModels/nn_gpu_full_train'
).fit(
    train_data=df_train_limpio,         # tus 7 millones
    tuning_data=df_val_limpio,          # tus 0.5 millones separados temporalmente
    time_limit=7200,                    # 2 horas (ajustable según complejidad)
    use_bag_holdout=False,              # obligatorio si usás tuning_data
    presets='medium',                   # no activa stacking ni mezcla datos
    hyperparameters={
        'NN_TORCH': [{
            'num_epochs': 40,
            'learning_rate': 0.005,
            'dropout_prob': 0.1,
            'batch_size': 2048,         # mayor batch para mejor uso de GPU
            'hidden_size': 256,         # red más potente
            'ag_args': {'name_suffix': 'GPU_Full'},
            'ag_args_fit': {'num_gpus': 1}
        }]
    }
)

Training data for TabularNeuralNetTorchModel has: 6730977 examples, 380 features (380 vector, 0 embedding)
Training on GPU
Neural network architecture:
EmbedNet(
  (main_block): Sequential(
    (0): Linear(in_features=380, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=256, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=256, out_features=256, bias=True)
    (10): ReLU()
    (11): Linear(in_features=256, out_features=1, bias=True)
  )
)
Training tabular neural network for up to 40 epochs...
Epoch 1 (Update 3286).	Train loss: 0.3348, Val mean_absolute_error: -0.2515, Best Epoch: 1
Epoch 2 (Update 6572).	Train loss: 0.3095, Val mean_absolute_error: -0.2561, Best Epoch: 1
Epoch 3 (Update 9858).	Train loss: 0.3022, Val mean_absolute_error: -0.2468,

In [ ]:

predictor = TabularPredictor(
    label='CLASE_DELTA_ZSCORE',
    problem_type='regression',
    eval_metric='mean_absolute_error',
    verbosity=4,
    path='AutogluonModels/nn_lightfast_test_15min'
).fit(
    train_data=df_entrenamiento_total,
    time_limit=900,  # 15 minutos = 900 segundos
    hyperparameters={
        'NN_TORCH': {
            'num_epochs': 5,              # reducir epochs acelera bastante
            'learning_rate': 0.01,
            'dropout_prob': 0.1,
            'batch_size': 1024,           # más batch size = mejor para GPU
            'hidden_size': 64,            # red más chica = más rápida
            'ag_args': {'name_suffix': 'GPU_15min'},
            'ag_args_fit': {'num_gpus': 1}
        }
    },
    presets='medium_quality',
    use_bag_holdout=False  # sin bagging ni tuning
)

In [ ]:
predictor = TabularPredictor(
    label='CLASE_DELTA_ZSCORE',
    problem_type='regression',
    eval_metric='mean_absolute_error',
    verbosity=4,
    path='AutogluonModels/nn_lightfast_test'
).fit(
    train_data=df_entrenamiento_total,
    time_limit=1800,  # 30 minutos
    hyperparameters={
        'NN_TORCH': {
            'num_epochs': 10,
            'learning_rate': 0.01,
            'dropout_prob': 0.1,
            'batch_size': 512,
            'hidden_size': 128,
            'ag_args': {'name_suffix': 'GPU_Basic'},
            'ag_args_fit': {'num_gpus': 1}
        }
    },
    presets='medium_quality',
    use_bag_holdout=False  # sin bagging, directo
)



In [ ]:
""" target_col = 'CLASE_DELTA_ZSCORE'  # ajustá si tu columna objetivo es otra

hyperparameters = {
    'NN_TORCH': [
        {'ag_args': {'name_suffix': 'LightFast'}}
    ]
}

predictor_nn_lightfast = TabularPredictor(
    label='CLASE_DELTA_ZSCORE',
    problem_type='regression',
    eval_metric='mean_absolute_error',
    verbosity=3,
    path='AutogluonModels/nn_lightfast_retrain'
).fit(
    train_data=df_train_limpio,
    tuning_data=df_val_limpio,
    time_limit=7200,
    # hyperparameters={
    #     'NN_TORCH': [{
    #         'num_epochs': 8,
    #         'learning_rate': 0.01,
    #         'dropout_prob': 0.1,
    #         'weight_decay': 1e-5,
    #         'batch_size': 256,
    #         'hidden_size': 128,
    #         'ag_args': {'name_suffix': 'NN_LightFast'},
    #         'ag_args_fit': {'num_gpus': 1}
    #     }]
    # },
    hyperparameters={
        'NN_TORCH': {}
    },
    hyperparameter_tune_kwargs='auto',
    presets='best_quality',
    use_bag_holdout=False
) """

In [ ]:
""" target_col = 'CLASE_DELTA_ZSCORE'  # Ajustá si es necesario

df_entrenamiento_total = pd.concat([df_train_limpio, df_val_limpio], axis=0)
del df_train_limpio, df_val_limpio
gc.collect()

predictor_nn_lightfast = TabularPredictor(
    label=target_col,
    problem_type='regression',
    eval_metric='mean_absolute_error',
    verbosity=3,
    path='AutogluonModels/nn_lightfast_retrain'
).fit(
    train_data=df_entrenamiento_total,
    #train_data=df_train_limpio,
    #tuning_data=df_val_limpio,
    time_limit=7200,
    presets='best_quality',
    hyperparameters={
        'NN_TORCH': {
            'ag_args': {'name_suffix': 'NN_LightFast'},
            'ag_args_fit': {'num_gpus': 1}
        }
    },
    hyperparameter_tune_kwargs='auto',
    use_bag_holdout=True
) """

In [ ]:

# === 3. Evaluación del modelo ===

lb = predictor_nn_lightfast.leaderboard(df_val_limpio, extra_metrics=['mean_absolute_error', 'median_absolute_error', 'r2'], silent=False)

# === 4. Importancia de características ===

feature_importance = predictor_nn_lightfast.feature_importance(df_val_limpio)

# === 5. Guardado del modelo ya está hecho con `path=...` ===

print("✅ Reentrenamiento completado y modelo guardado en 'AutogluonModels/nn_lightfast_retrain'")


✅ Paso 1: Calcular errores de predicción

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Predicciones y error absoluto
df_val['y_true'] = df_val['CLASE_DELTA_ZSCORE']
df_val['y_pred'] = predictor.predict(df_val)
df_val['error_abs'] = abs(df_val['y_true'] - df_val['y_pred'])
df_val['error_signed'] = df_val['y_pred'] - df_val['y_true']


📊 Paso 2: Histogramas de error absoluto y error signado

In [ ]:
plt.figure(figsize=(10, 4))
sns.histplot(df_val['error_abs'], bins=50, kde=True)
plt.title('Distribución del Error Absoluto')
plt.xlabel('|y_pred - y_true|')
plt.ylabel('Frecuencia')
plt.show()

plt.figure(figsize=(10, 4))
sns.histplot(df_val['error_signed'], bins=50, kde=True)
plt.title('Distribución del Error Signado')
plt.xlabel('y_pred - y_true')
plt.ylabel('Frecuencia')
plt.axvline(0, color='red', linestyle='--')
plt.show()


📈 Paso 3: Error vs. Valor real (dispersión)


In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df_val['y_true'], y=df_val['error_signed'], alpha=0.3)
plt.axhline(0, color='red', linestyle='--')
plt.title('Error Signado vs. Valor Real')
plt.xlabel('Valor real')
plt.ylabel('Error (pred - real)')
plt.show()


🧩 Paso 4: Promedio de error por grupo 

In [ ]:
top_errores_producto = df_val.groupby('PRODUCT_ID')['error_abs'].mean().sort_values(ascending=False).head(20)
top_errores_producto.plot(kind='bar', figsize=(10,4), title='Top 20 PRODUCT_ID con mayor error promedio')
plt.ylabel('Error Absoluto Medio')
plt.show()


In [ ]:
top_errores_producto = df_val.groupby('CUSTOMER_ID')['error_abs'].mean().sort_values(ascending=False).head(20)
top_errores_producto.plot(kind='bar', figsize=(10,4), title='Top 20 CUSTOMER_ID con mayor error promedio')
plt.ylabel('Error Absoluto Medio')
plt.show()

In [ ]:
df_full = pd.concat([df_train_limpio, df_val_limpio], ignore_index=True)
del df_train_limpio, df_val_limpio
gc.collect()
predictor_full = predictor_nn_lightfast.refit_full(train_data=df_full)
predictor_full.save("AutogluonModels/nn_lightfast_full")

In [ ]:
""" # Combinar entrenamiento + validación
df_full = pd.concat([df_train, df_val], axis=0)
del df_train, df_val
gc.collect() """

In [ ]:
# # Reentrenar el mejor modelo con TODOS los datos disponibles
# predictor_full = predictor.refit_full(train_data=df_full)

In [ ]:
# Verificar los modelos disponibles (el mejor ahora tiene el sufijo '_FULL')
print("Modelos disponibles luego del refit completo:")
print(predictor.leaderboard(silent=True)['model'].tolist())
# Eliminar modelos intermedios para liberar espacio
predictor.delete_models(models_to_keep='best', dry_run=False)

# Confirmar que solo queda el modelo reentrenado
print("\nModelos restantes después de eliminar los intermedios:")
print(predictor.leaderboard(silent=True)['model'].tolist())

# (Opcional) Guardar el predictor final si querés usarlo luego sin volver a cargar todo
predictor.save('./data/modelo_final_autogluon')

# ---  Liberar memoria ---
del df_full
gc.collect()


In [ ]:
# Cargo los datos sobre los que quiero hacer predicciones
df_pred_full = pd.read_parquet('./data/l_vm_completa_normalizada_fe.parquet', engine='fastparquet')
# Dejo solo los datos del periodo 201910 y que A_PREDECIR sea True
# Filtrar solo los datos del periodo 201910 y donde A_PREDECIR sea True
df_pred_full = df_pred_full[
    (df_pred_full['PERIODO'] == 201910) & (df_pred_full['A_PREDECIR'] == True)
].drop(columns=['CLASE_ZSCORE', 'CLASE_DELTA_ZSCORE'])

In [ ]:
# Realizar las predicciones usando el predictor original
predictions = predictor.predict(df_pred_full)
# Agregar las predicciones al DataFrame original
df_pred_full['CLASE_DELTA_ZSCORE'] = predictions

In [ ]:
# Imprimir la lista de columas del DataFrame con las predicciones
print("Columnas del DataFrame con las predicciones:")
print(df_pred_full.columns.tolist())

In [ ]:
# Dernormalizar la columna CLASE_DELTA_ZSCORE
df_pred_full['CLASE_DELTA'] = df_pred_full['CLASE_DELTA_ZSCORE'] * df_pred_full['CLASE_DELTA_STD'] + df_pred_full['CLASE_DELTA_MEAN']
df_pred_full['TN'] = df_pred_full['TN_ZSCORE'] * df_pred_full['TN_STD'] + df_pred_full['TN_MEAN']
# Agregar la columna TN_PREDICT que sea la suma de TN y CLASE_DELTA y si es menor que cero, poner cero
df_pred_full['TN_PREDICT'] = df_pred_full['TN'] + df_pred_full['CLASE_DELTA']
df_pred_full['TN_PREDICT'] = df_pred_full['TN_PREDICT'].clip(lower=0)

In [ ]:
# Generar Dataframe que contenga por cada PRODUCT_ID la suma de TN_PREDICT
df_final = df_pred_full.groupby('PRODUCT_ID').agg({'TN_PREDICT': 'sum'}).reset_index()
df_final = df_final.rename(columns={'PRODUCT_ID': 'product_id', 'TN_PREDICT': 'tn'})
# Guardar el DataFrame df_final en un archivo CSV
df_final.to_csv('./modelos/autoglun_normalizando_clase_delta.csv', index=False)
df_final.shape

# Para instalar AutoGluon con soporte para `autogluon.core.space` en conda, ejecuta:
# 
# conda install -c conda-forge autogluon
# 
# O si prefieres usar pip dentro de tu entorno conda:
# 
# pip install autogluon
# 
# Luego podrás usar:
# from autogluon.core import space as ag